In [1]:
pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 22.0 MB/s eta 0:00:00


In [2]:
import transformers
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/final_data_with_hashtags_2.csv')
df = df.drop(columns = ['Unnamed: 0'])
df.dropna(inplace = True)
df.head()

,Caption,Tags
0,"""Falling for the fairytale 🌹🥀",#beautyandthebeast
1,"""Falling for the fairytale 🌹🥀",#fairytalelove
2,"""Falling for the fairytale 🌹🥀",#happilyeverafter
3,"""A tale as old as time 🕰️💕",#disneylover
4,"""A tale as old as time 🕰️💕",#fairytalelife


In [9]:
train_df, val_df = train_test_split(df, test_size = 0.1)
val_df, test_df = train_test_split(val_df, test_size = 0.5)

In [10]:
def process(row): 
    row['Caption'] = '<SOS> ' + row['Caption'] + ' <EOS>'
    row['Tags'] = '<SOT> ' + row['Tags'] + ' <EOT>'
    return row

In [11]:
train = train_df.apply(lambda x: process(x), axis = 1)
val = val_df.apply(lambda x: process(x), axis = 1)
test = test_df.apply(lambda x: process(x), axis = 1)

In [ ]:
train

,Caption,Tags
12627,"<SOS> ""Probiotic power! 🌟🥬 <EOS>",<SOT> #healthyliving <EOT>
27439,"<SOS> ""Stylish in Multicolor! 🌈👗 <EOS>",<SOT> #ActorStyle <EOT>
27840,"<SOS> Ready for anything, anytime, anywhere 💥🚀...",<SOT> #preparednes <EOT>
2477,<SOS> The ultimate in luxury living! 🏢🏙️ <EOS>,<SOT> #ResidentialLiving <EOT>
11423,"<SOS> ""Coastal wanderlust! Embarking on a scen...",<SOT> #ScenicDrive <EOT>
...,...,...
29309,<SOS> The hustle and bustle of city life 🌆🚶‍♂️...,<SOT> #urbanwalk <EOT>
26686,<SOS> A cozy abode just for you 🛋🏡🌅 <EOS>,<SOT> #RentalLif <EOT>
19637,"<SOS> ""Pioneering the way for women in educati...",<SOT> #WomenEmpowerment <EOT>
499,"<SOS> Practice makes perfect, and this cricket...",<SOT> #GameO <EOT>


In [12]:
train = Dataset.from_pandas(train.reset_index(drop = True))
val = Dataset.from_pandas(val.reset_index(drop = True))
test = Dataset.from_pandas(test.reset_index(drop = True))

In [14]:
max_len = 0
for pair in train:
  max_len = max(max_len, len(pair['Caption'].split()), len(pair['Tags'].split()))

for pair in val:
  max_len = max(max_len, len(pair['Caption'].split()), len(pair['Tags'].split()))

for pair in test:
  max_len = max(max_len, len(pair['Caption'].split()), len(pair['Tags'].split()))

max_len

32

In [15]:
self_tokenizer = Tokenizer(filters='', oov_token='<OOV>')
self_tokenizer.fit_on_texts(train['Caption'])
self_tokenizer.fit_on_texts(train['Tags'])
self_tokenizer.fit_on_texts(val['Caption'])
self_tokenizer.fit_on_texts(val['Tags'])
self_tokenizer.fit_on_texts(test['Caption'])
self_tokenizer.fit_on_texts(test['Tags'])

In [34]:
model_checkpoint = "t5-base"
# model_checkpoint = '/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/t5_hashtag_model_10_epochs_2'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
new_tokens = set(self_tokenizer.word_index.keys()) - set(tokenizer.vocab.keys())

In [ ]:
import string
new_tokens_clean = set()
for token in new_tokens:
  new_tokens_clean.add(token.translate(str.maketrans('', '', string.punctuation)))

In [ ]:
temp = set()
new_tokens = set()
for tokens in new_tokens_clean:
  if '<SOS>'  in tokens or '<EOS>' in tokens:
    continue
  if tokens.isalnum() == False:
    temp.add(tokens)
  else:
    new_tokens.add(tokens)

In [ ]:
new_tokens

In [ ]:
tokenizer.add_tokens(list(new_tokens))

0

In [ ]:
special_tokens = ['<SOS>', '<EOS>', '<EOT>', '<SOT>', 'generate tags']
tokenizer.add_tokens(special_tokens, special_tokens=True)

0

In [4]:
prefix = "generate tags: "

def preprocess_function(examples):
  # print(examples)
  inputs = [prefix + example for example in examples['Caption']]
  targets = [example for example in examples['Tags']]
  # print(inputs)
  # print(targets)
  model_inputs = tokenizer(inputs, max_length = max_len, truncation = True, padding = True, text_target = targets, return_tensors = 'pt')
  # print(type(model_inputs))
  return model_inputs

In [ ]:
train = train.map(preprocess_function, batched = True)
val = val.map(preprocess_function, batched = True)

Map:   0%|          | 0/27855 [00:00<?, ? examples/s]

Map:   0%|          | 0/1548 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model= model_checkpoint)

In [37]:
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-common_gen")
model.resize_token_embeddings(len(tokenizer))
# model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/t5_hashtag_model_10_epochs").cuda()

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Embedding(32100, 768)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/t5_hashtag",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset = val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.248842
2,1.929400,1.075978
3,1.169300,1.015651
4,1.036500,0.981713
5,0.967000,0.953678
6,0.912100,0.939908
7,0.870300,0.926782
8,0.870300,0.922544
9,0.841200,0.917841
10,0.818400,0.917670


TrainOutput(global_step=4360, training_loss=1.04657548676937, metrics={'train_runtime': 1753.1982, 'train_samples_per_second': 158.881, 'train_steps_per_second': 2.487, 'total_flos': 1.0601576275968e+16, 'train_loss': 1.04657548676937, 'epoch': 10.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/t5_hashtag_model_10_epochs_2')

In [5]:
model_2 = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/t5_hashtag_model_10_epochs_2")

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [6]:
model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/t5_hashtag_model_10_epochs")

In [38]:
def gen_tag(s):
    temp = prefix + s
    temp_token = tokenizer(temp, max_length = max_len, truncation = True, padding = True, return_tensors = 'pt').input_ids
    return model.generate(temp_token)

In [17]:
no = 75
temp = prefix + test['Caption'][no]
# temp = prefix + 'vector illustration of a colorful shiny lamp for person with brown background'
temp_token = tokenizer(temp, max_length = max_len, truncation = True, padding = True, return_tensors = 'pt').input_ids
temp = model.generate(temp_token)
temp_2 = model_2.generate(temp_token)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [18]:
test['Caption'][no]

'<SOS> "Passion for photography - a young boy\'s excitement shines through as he captures a moment with his camera 📷😃  <EOS>'

In [19]:
tokenizer.batch_decode(temp)

['<pad><SOT> #P<EOT></s>']

In [20]:
tokenizer.batch_decode(temp_2)

['<pad><SOT> #Pas sionForPhotography #PhotographyMonth<EOT></s>']

In [39]:
from tqdm.auto import tqdm
df_test = pd.DataFrame(test, columns = ['Caption', 'Tags'])
tqdm.pandas()
df_test['Generated_tags'] = df_test.progress_apply(lambda x: gen_tag(x['Caption']), axis = 1)

  0%|          | 0/1548 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [40]:
df_test['Generated_tags'] = df_test.progress_apply(lambda x: tokenizer.batch_decode(x['Generated_tags']), axis = 1)

  0%|          | 0/1548 [00:00<?, ?it/s]

In [41]:
df_test

,Caption,Tags,Generated_tags
0,"<SOS> ""Fall Decor Goals! 🍁🧡 <EOS>",<SOT> #WoodenBackground <EOT>,"[<pad> tagged with <unk>SOS> ""Fall Goals!</s>]"
1,"<SOS> ""Butterfly dance! 🦋🔳 <EOS>",<SOT> #AbstractButterflies <EOT>,[<pad> a butterfly dances with a twig and a <u...
2,"<SOS> ""On Your Mark! 🏃‍♂️🌳 <EOS>",<SOT> #ReadyToRace <EOT>,"[<pad> <unk> <unk> <unk>EOS> tags: ""On Your Ma..."
3,<SOS> The love of your life is the one who mak...,<SOT> #soulmate <EOT>,[<pad> <unk> <unk> <unk> <unk> <unk> <unk> <un...
4,"<SOS> ""A-List Appearance! 🎬🌟 <EOS>",<SOT> #MovieStarStatus <EOT>,"[<pad> <unk>SOS> tag: ""A-List Appearance! <unk..."
...,...,...,...
1543,<SOS> Celebrating life and death with this stu...,<SOT> paint <EOT>,[<pad> tagged with <unk>SOS> a beautiful woman...
1544,"<SOS> ""Exploring the world with a map in hand ...",<SOT> #Wanderlust <EOT>,[<pad> a bus at a station with a map in hand.<...
1545,"<SOS> ""Sand, sea, and paws! <EOS>",<SOT> #BeachDays <EOT>,"[<pad> tags: <unk>SOS> ""Sand, sea, and paws!</s>]"
1546,"<SOS> ""Wedding fashion - detailed look at what...",<SOT> #WeddingAttire <EOT>,[<pad> a detailed look at what the bride wore ...


In [42]:
df_test.to_pickle('/content/drive/MyDrive/Information Retrieval Project/FINAL SUBMISSION FILES/t5_hashtags_not_trained.pkl')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
epochs = np.arange(1,11)
train_loss = np.array([1.929400, 1.929400, 1.169300, 1.036500, 0.967000, 0.912100, 0.870300, 0.870300, 0.841200, 0.814800])
val_loss = np.array([1.248842, 1.075978, 1.015651, 0.981713, 0.953678, 0.939908, 0.926782, 0.922544, 0.917841, 0.917670])

plt.plot(epochs, train_loss, label = 'Train_loss')
plt.plot(epochs, val_loss, label = 'Val loss')
plt.legend()
plt.title("T5 fine-tuned for Hashtag Loss curves")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()